# Tutorial 5  ₍ᐢ･⚇･ᐢ₎و

In [1]:
import pandas as pd
import numpy as np

# Delete the 'male' column
insur = pd.read_csv('insurance_data_cleaned.csv', index_col='PatientID')
del insur['male']
insur.head()

,age,gender,bmi,bloodpressure,diabetic,children,smoker,region,claim
PatientID,,,,,,,,,
1,39,male,23.2,91,Yes,0,No,southeast,1121.87
2,24,male,30.1,87,No,0,No,southeast,1131.51
8,19,male,41.1,100,No,0,No,northwest,1146.80
9,20,male,43.0,86,No,0,No,northwest,1149.40
10,30,male,53.1,97,No,0,No,northwest,1163.46


# Groupby and related descriptions

In [2]:
# Extract total claim conditional on region
insur.groupby('region')['claim'].sum()

region
northeast    3901369.33
northwest    4069006.54
southeast    5783789.60
southwest    3995062.77
Name: claim, dtype: float64

In [3]:
# summarize the mean of claim conditional on region, remain 2 decimals
insur.groupby('region')['claim'].mean().round(2)

region
northeast    16889.04
northwest    11794.22
southeast    13085.50
southwest    12723.13
Name: claim, dtype: float64

In [4]:
# summarize standard deviations of claim conditional on region, and sort
insur.groupby('region')['claim'].std().sort_values()

region
northwest    11036.722585
northeast    11578.101476
southwest    11578.518764
southeast    13179.731930
Name: claim, dtype: float64

In [5]:
# Conditional on gender, get both means and medians for columns bmi and bloodpressure
insur.groupby('gender')[['bmi', 'bloodpressure']].agg(['mean', 'median']).round(2)

bmi        bloodpressure       
         mean median          mean median
gender                                   
female  30.38   30.1         94.01     93
male    30.93   30.7         94.36     92

# Data-Cleaning: Remove outliers

### Method 1: Z-score

In [6]:
from scipy.stats import zscore
z = zscore(insur['claim'])
z

array([-1.00812075, -1.00732438, -1.00606128, ...,  3.85756855,
        4.06999765,  4.16727586])

In [7]:
insur_outliers = insur[np.abs(z) > 3]
insur_outliers

,age,gender,bmi,bloodpressure,diabetic,children,smoker,region,claim
PatientID,,,,,,,,,
1334,44,male,36.4,127,No,1,Yes,southwest,51194.56
1335,43,male,32.8,125,No,0,Yes,southwest,52590.83
1336,44,female,35.5,88,Yes,0,Yes,northwest,55135.40
1337,59,female,38.1,120,No,1,Yes,northeast,58571.07
1338,30,male,34.5,91,Yes,3,Yes,northwest,60021.40
1339,37,male,30.4,106,No,0,Yes,southeast,62592.87
1340,30,female,47.4,101,No,0,Yes,southeast,63770.43


In [8]:
insur_no_outliers = insur[np.abs(z) <= 3]
insur_no_outliers

,age,gender,bmi,bloodpressure,diabetic,children,smoker,region,claim
PatientID,,,,,,,,,
1,39,male,23.2,91,Yes,0,No,southeast,1121.87
2,24,male,30.1,87,No,0,No,southeast,1131.51
8,19,male,41.1,100,No,0,No,northwest,1146.80
9,20,male,43.0,86,No,0,No,northwest,1149.40
10,30,male,53.1,97,No,0,No,northwest,1163.46
...,...,...,...,...,...,...,...,...,...
1329,45,male,42.1,117,No,1,Yes,southeast,48675.52
1330,52,female,37.7,109,Yes,0,Yes,southwest,48824.45
1331,25,female,38.1,111,No,0,Yes,southeast,48885.14


### Method 2: Inter Quartile Range

In [9]:
q1 = np.percentile(insur['claim'], 25)
q3 = np.percentile(insur['claim'], 75)
iqr = q3 - q1

upper = insur['claim'] >= (q3 + 1.5 * iqr)
lower = insur['claim'] <= (q1 - 1.5 * iqr)

insur_outliers = insur[(upper | lower)]
insur_outliers

,age,gender,bmi,bloodpressure,diabetic,children,smoker,region,claim
PatientID,,,,,,,,,
1206,20,male,34.9,124,Yes,0,Yes,southwest,34828.65
1207,46,female,31.4,111,No,0,Yes,southwest,34838.87
1208,22,male,22.9,80,No,0,Yes,northeast,35069.37
1209,23,male,28.5,106,No,0,Yes,northeast,35147.53
1210,32,female,26.8,107,No,1,No,southwest,35160.13
...,...,...,...,...,...,...,...,...,...
1336,44,female,35.5,88,Yes,0,Yes,northwest,55135.40
1337,59,female,38.1,120,No,1,Yes,northeast,58571.07
1338,30,male,34.5,91,Yes,3,Yes,northwest,60021.40


In [10]:
# ~
insur_no_outliers = insur[~(upper | lower)]
insur_no_outliers

,age,gender,bmi,bloodpressure,diabetic,children,smoker,region,claim
PatientID,,,,,,,,,
1,39,male,23.2,91,Yes,0,No,southeast,1121.87
2,24,male,30.1,87,No,0,No,southeast,1131.51
8,19,male,41.1,100,No,0,No,northwest,1146.80
9,20,male,43.0,86,No,0,No,northwest,1149.40
10,30,male,53.1,97,No,0,No,northwest,1163.46
...,...,...,...,...,...,...,...,...,...
1201,40,male,32.7,98,No,0,Yes,southwest,34472.84
1202,45,male,33.5,81,No,0,Yes,northeast,34617.84
1203,32,male,31.7,125,No,0,Yes,southeast,34672.15


### Method 3: Replace extreme values (> 99% or < 1%) with 99 percentile or 1 percentile

In [11]:
upper=np.percentile(insur['claim'], 99)
lower=np.percentile(insur['claim'], 1)

insur_replace= insur.copy()

for value in insur_replace['claim'].values:
    if value >= upper:
            insur_replace[insur_replace['claim'].values==value].replace([value],upper)
    elif value <= lower:
             insur_replace[insur_replace['claim'].values==value].replace([value],lower)
insur_replace

,age,gender,bmi,bloodpressure,diabetic,children,smoker,region,claim
PatientID,,,,,,,,,
1,39,male,23.2,91,Yes,0,No,southeast,1121.87
2,24,male,30.1,87,No,0,No,southeast,1131.51
8,19,male,41.1,100,No,0,No,northwest,1146.80
9,20,male,43.0,86,No,0,No,northwest,1149.40
10,30,male,53.1,97,No,0,No,northwest,1163.46
...,...,...,...,...,...,...,...,...,...
1336,44,female,35.5,88,Yes,0,Yes,northwest,55135.40
1337,59,female,38.1,120,No,1,Yes,northeast,58571.07
1338,30,male,34.5,91,Yes,3,Yes,northwest,60021.40


# Finance Data - Practice of Groupby

In [12]:
import yfinance as yf

tickers = ['AAPL', 'BA', 'MCD', 'WMT']
df = yf.download(tickers, start='2021-01-01', end='2022-12-31', interval='1d', group_by='ticker')
df.dropna(inplace=True)
df

[*********************100%***********************]  4 of 4 completed


AAPL                                                  \
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2021-01-04  133.520004  133.610001  126.760002  129.410004  127.164169   
2021-01-05  128.889999  131.740005  128.429993  131.009995  128.736374   
2021-01-06  127.720001  131.050003  126.379997  126.599998  124.402908   
2021-01-07  128.360001  131.630005  127.860001  130.919998  128.647919   
2021-01-08  132.429993  132.630005  130.229996  132.050003  129.758362   
...                ...         ...         ...         ...         ...   
2022-12-23  130.919998  132.419998  129.639999  131.860001  131.127075   
2022-12-27  131.380005  131.410004  128.720001  130.029999  129.307220   
2022-12-28  129.669998  131.029999  125.870003  126.040001  125.339409   
2022-12-29  127.989998  130.479996  127.730003  129.610001  128.889557   
2022-12-30  128.410004  129.949997  127.430000  129.929993  129.207779   

                              MCD                                      ...  \
               Volume        Open        High         Low       Close  ...   
Date                                                                   ...   
2021-01-04  143301900  214.490005  214.720001  208.220001  210.220001  ...   
2021-01-05   97664900  210.179993  211.949997  209.619995  211.479996  ...   
2021-01-06  155088000  211.300003  211.710007  209.029999  211.000000  ...   
2021-01-07  109578200  213.220001  213.220001  210.559998  211.979996  ...   
2021-01-08  105158200  212.899994  216.119995  212.229996  215.869995  ...   
...               ...         ...         ...         ...         ...  ...   
2022-12-23   63814900  265.899994  268.350006  264.790009  267.570007  ...   
2022-12-27   69007800  268.660004  268.869995  266.600006  266.839996  ...   
2022-12-28   85438400  268.000000  268.140015  265.070007  265.109985  ...   
2022-12-29   75703700  265.940002  267.809998  264.880005  265.929993  ...   
2022-12-30   77034200  265.200012  265.380005  261.399994  263.529999  ...   

                   WMT                                            BA  \
                   Low       Close   Adj Close    Volume        Open   
Date                                                                   
2021-01-04  144.279999  146.529999  139.870651  10727400  210.000000   
2021-01-05  144.490005  145.750000  139.126114   8832700  204.740005   
2021-01-06  145.009995  146.660004  139.994751   7306300  210.220001   
2021-01-07  146.039993  146.649994  139.985184   6846000  213.389999   
2021-01-08  145.809998  146.630005  139.966125   8159400  213.610001   
...                ...         ...         ...       ...         ...   
2022-12-23  142.279999  143.770004  141.610046   3182500  188.250000   
2022-12-27  143.190002  143.809998  141.649460   3348200  189.580002   
2022-12-28  140.910004  141.289993  139.167313   5082100  189.320007   
2022-12-29  141.750000  142.149994  140.014389   3057300  189.000000   
2022-12-30  140.809998  141.789993  139.659790   3835300  187.500000   

                                                                      
                  High         Low       Close   Adj Close    Volume  
Date                                                                  
2021-01-04  210.199997  202.490005  202.720001  202.720001  21225600  
2021-01-05  213.350006  204.600006  211.630005  211.630005  19338300  
2021-01-06  215.610001  209.339996  211.029999  211.029999  16202200  
2021-01-07  216.600006  211.779999  212.710007  212.710007  14474100  
2021-01-08  214.100006  208.160004  209.899994  209.899994  14144000  
...                ...         ...         ...         ...       ...  
2022-12-23  189.429993  184.720001  189.059998  189.059998   3983600  
2022-12-27  192.440002  187.619995  189.399994  189.399994   5014800  
2022-12-28  191.320007  187.429993  188.380005  188.380005   4426300  
2022-12-29  190.250000  186

In [13]:
aapl = df['AAPL'].reset_index()
ba = df['BA'].reset_index()
mcd = df['MCD'].reset_index()
wmt = df['WMT'].reset_index()

In [14]:
aapl.insert(1, 'Ticker', 'AAPL')
ba.insert(1, 'Ticker', 'BA')
mcd.insert(1, 'Ticker', 'MCD')
wmt.insert(1, 'Ticker', 'WMT')

In [15]:
aapl

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,AAPL,133.520004,133.610001,126.760002,129.410004,127.164169,143301900
1,2021-01-05,AAPL,128.889999,131.740005,128.429993,131.009995,128.736374,97664900
2,2021-01-06,AAPL,127.720001,131.050003,126.379997,126.599998,124.402908,155088000
3,2021-01-07,AAPL,128.360001,131.630005,127.860001,130.919998,128.647919,109578200
4,2021-01-08,AAPL,132.429993,132.630005,130.229996,132.050003,129.758362,105158200
...,...,...,...,...,...,...,...,...
498,2022-12-23,AAPL,130.919998,132.419998,129.639999,131.860001,131.127075,63814900
499,2022-12-27,AAPL,131.380005,131.410004,128.720001,130.029999,129.307220,69007800
500,2022-12-28,AAPL,129.669998,131.029999,125.870003,126.040001,125.339409,85438400
501,2022-12-29,AAPL,127.989998,130.479996,127.730003,129.610001,128.889557,75703700


In [16]:
# Create a concated dataframe
stocks = pd.concat([aapl, ba, mcd, wmt], ignore_index=True)
stocks

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,AAPL,133.520004,133.610001,126.760002,129.410004,127.164169,143301900
1,2021-01-05,AAPL,128.889999,131.740005,128.429993,131.009995,128.736374,97664900
2,2021-01-06,AAPL,127.720001,131.050003,126.379997,126.599998,124.402908,155088000
3,2021-01-07,AAPL,128.360001,131.630005,127.860001,130.919998,128.647919,109578200
4,2021-01-08,AAPL,132.429993,132.630005,130.229996,132.050003,129.758362,105158200
...,...,...,...,...,...,...,...,...
2007,2022-12-23,WMT,143.050003,143.800003,142.279999,143.770004,141.610046,3182500
2008,2022-12-27,WMT,144.039993,144.449997,143.190002,143.809998,141.649460,3348200
2009,2022-12-28,WMT,143.470001,143.669998,140.910004,141.289993,139.167313,5082100
2010,2022-12-29,WMT,142.139999,143.020004,141.750000,142.149994,140.014389,3057300


In [17]:
stocks['Return'] = stocks.groupby('Ticker')['Close'].pct_change()
stocks.head()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Return
0,2021-01-04,AAPL,133.520004,133.610001,126.760002,129.410004,127.164169,143301900,NaN
1,2021-01-05,AAPL,128.889999,131.740005,128.429993,131.009995,128.736374,97664900,0.012364
2,2021-01-06,AAPL,127.720001,131.050003,126.379997,126.599998,124.402908,155088000,-0.033662
3,2021-01-07,AAPL,128.360001,131.630005,127.860001,130.919998,128.647919,109578200,0.034123
4,2021-01-08,AAPL,132.429993,132.630005,130.229996,132.050003,129.758362,105158200,0.008631


In [18]:
# pivot tables
pivot_return = stocks.pivot(index='Date', columns='Ticker', values='Return')
pivot_return.head()

Ticker,AAPL,BA,MCD,WMT
Date,,,,
2021-01-04,NaN,NaN,NaN,NaN
2021-01-05,0.012364,0.043952,0.005994,-0.005323
2021-01-06,-0.033662,-0.002835,-0.002270,0.006244
2021-01-07,0.034123,0.007961,0.004645,-0.000068
2021-01-08,0.008631,-0.013211,0.018351,-0.000136


In [19]:
stocks.groupby('Ticker')['Return'].agg(['mean', 'min', 'max'])

,mean,min,max
Ticker,,,
AAPL,0.000196,-0.058680,0.088975
BA,0.000218,-0.104701,0.094630
MCD,0.000513,-0.048683,0.040876
WMT,0.000034,-0.113758,0.065395
